In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelos lineales generalizados: regresion de Poisson
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.discrete.discrete_model import Poisson
from statsmodels.discrete.diagnostic import PoissonDiagnostic

data = {
    'Embarazos reportados': [0, 1, 2, 3, 4],
    'n': [1011, 190, 32, 6, 2],
    '%': [81.5, 15.3, 2.6, 0.5, 0.2]
}

df = pd.DataFrame(data)
print(df)

# Gráfico de barras
plt.figure(figsize=(4, 4))
plt.bar(df['Embarazos reportados'], df['%'], color='brown')
plt.title('Frecuencia de Embarazos Adolescentes')
plt.xlabel('Embarazos reportados')
plt.ylabel('Porcentaje (%)')
plt.xticks(df['Embarazos reportados'])
plt.grid(True)
plt.show()

In [ ]:
# Cargar el conjunto de datos:
url = "https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/crimenes.csv"
datos = pd.read_csv(url)
print(datos.head())
'''
Ejemplo de motivación: crimenes en instituciones postsecundarias Estados Unidos:
matriculados = personas matriculadas
tipo = colegio (C) o universidad (U)
nv = número de delitos violentos cometidos en la universidad o colegio
nvrate = número de delitos violentos por cada 1000 estudiantes
matriculados = personas matriculadas, en miles
region = región del país (C = Central, 
                         MO = Medio Oeste, 
                         NE = Noreste, 
                         SE = Sudeste, 
                         SO = Suroeste y 
                         O = Oeste)
Fuente: US Department of Education
'''

In [ ]:
# Plot the distribution of the number of violent crimes (nv)
plt.figure(figsize=(5, 6))
hist = sns.histplot(datos['nv'], bins=15, kde=True, color='limegreen')
plt.title('Distribución de Crímenes Violentos en EE.UU.')
plt.xlabel('Número de Crímenes Violentos')
plt.ylabel('Frecuencia')
plt.grid(True)
# Ajustando las marcas de los ticks en el eje x al centro de las barras:
bin_edges = hist.patches[0].get_bbox().bounds
bin_centers = 0.45 * (hist.patches[0].get_width()) + np.array([p.get_x() for p in hist.patches])
plt.xticks(bin_centers, [f'{int(b)}' for b in bin_centers])
plt.show()

In [ ]:
# Crear el gráfico de nv contra matriculados
plt.figure(figsize=(5, 6))
sns.scatterplot(data=datos, x='matriculados', y='nv')
sns.regplot(data=datos, x='matriculados', y='nv', scatter=False, 
            logx=True, ci=None, line_kws={"color": "red"})
plt.title("Número de Crímenes vs Matriculados")
plt.xlabel("Matriculados")
plt.ylabel("Número de Crímenes")
plt.show()

# Crear el gráfico de nv contra region
plt.figure(figsize=(5, 6))
sns.boxplot(data=datos, x='region', y='nv')
plt.title("Número de Crímenes por Región")
plt.xlabel("Región USA")
plt.ylabel("Número de Crímenes")
plt.show()

In [ ]:
# Ajustando los modelos Poisson
modelo_completo = smf.poisson('nv ~ matriculados + tipo + region', data=datos).fit()
print(modelo_completo.summary())

In [ ]:
# Poisson con offset
datos['log_matriculados'] = np.log(datos['matriculados'])

# Ajustando el modelo Poisson con offset de log(matriculados)
modelo_completo = smf.poisson('nv ~ tipo + region', data=datos, offset=datos['log_matriculados']).fit()
print(modelo_completo.summary())

In [ ]:
# Ajustar un modelo quasi-Poisson
modelo_completo = smf.glm('nv ~ tipo + region',
                          data=datos,
                          offset=datos['log_matriculados'],
                          family=sm.families.Poisson()).fit(scale='X2')
print(modelo_completo.summary())

In [ ]:
# Poisson con offset y MVCV robusta
modelo_completo = smf.poisson('nv ~ tipo + region', 
                              data=datos, 
                              offset=datos['log_matriculados']).fit(cov_type='HC0')
print(modelo_completo.summary())

In [ ]:
# Binomial negativo con offset y MVC robusta
modelo_completo = smf.negativebinomial('nv ~ tipo + region', 
                                       data=datos, offset=datos['log_matriculados']).fit(cov_type='HC0')
print(modelo_completo.summary())

In [ ]:
# Modelo binomial con inflacion de zeros y MVC robusta
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
# fórmula del modelo
formula = 'nv ~ tipo + region'

# variables para la inflación de ceros como matriz de diseño
exog_infl = datos[['tipo', 'region']]
exog_infl = sm.add_constant(exog_infl)  # Agregar intercepto
exog_infl = exog_infl.to_numpy()  # Convertir a array de numpy

# Ajustar el modelo Zero-Inflated Negative Binomial con offset
modelo_completo = ZeroInflatedNegativeBinomialP.from_formula(formula, 
                                                             data=datos, 
                                                             exog_infl=exog_infl,  # Modelo para la inflación de ceros
                                                             offset=np.log(datos['matriculados'])).fit(cov_type='HC0')

print(modelo_completo.summary())

In [ ]:
from statsmodels.discrete.count_model import ZeroInflatedPoisson
# Asegúrate de que las variables categóricas estén codificadas adecuadamente
formula = 'nv ~ tipo + region'

# variables para la inflación de ceros como matriz de diseño
exog_infl = datos[['tipo', 'region']]
exog_infl = sm.add_constant(exog_infl)  # Agregar intercepto
exog_infl = exog_infl.to_numpy()  # Convertir a array de numpy

# Ajustar el modelo Zero-Inflated Poisson con offset
modelo_completo = ZeroInflatedPoisson.from_formula(formula, 
                                                   data=datos, 
                                                   exog_infl=exog_infl,  # Modelo para la inflación de ceros
                                                   offset=datos['log_matriculados']).fit(cov_type='HC0')
print(modelo_completo.summary())
